# EDA로 넷플릭스 알아보기

![](https://i.cbc.ca/1.4925337.1543629829!/cpImage/httpImage/image.jpg_gen/derivatives/16x9_780/netflix-prices-20181129.jpg)

넷플릭스 타이틀에 대해서 알아보자

우선 kaggle에 있는 데이터를 가져 와서 추가하고

탐사적 데이터 분석(EDA, Exploratory Data Analysis)으로  넷플릭스 분석해 보는 순으로 진행...

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

우선 pandas를 import하고 시작해 보자 

In [ ]:
import pandas as pd

넷플릭스 타이틀 정보 읽어 오기

In [ ]:
df_all = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

읽은 데이터 형태가 어떤지 체크

In [ ]:
df_all.head()

컬럼은...

In [ ]:
for col in df_all.columns:
    print(col) 

이 데이터 프레임을 사용해서 분석 시작

타이틀의 갯수는

In [ ]:
print(df_all.shape[0], '편')

연도별로 출시된 타이틀 수를 뽑아 보자

타이틀이 가장 많이 출시된 해는

In [ ]:
df_year = df_all['release_year'].value_counts()

print( df_year.idxmax(), '년')

그때 타이틀 수는

In [ ]:
print(df_year.max(), '편')

밀레니엄부터 19년까지만 뽑아서 차트를 그려 보면

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

%matplotlib inline

def draw_bar_chart(data, title):
    plt.rcParams['axes.unicode_minus'] = False
    fontpath = '../input/nanumgothic/NanumGothic.ttf'
    fontprop = font_manager.FontProperties(fname=fontpath, size=12)

    fig, ax = plt.subplots(1,1, figsize=(18, 6))

    ax.bar(data.index, data.values, width=0.55, 
           edgecolor='darkgray', color='#d4dddd',
           linewidth=0.7)

    for i in data.index:
        ax.annotate(f"{data[i]}", 
                       xy=(i, data[i]),
                       va = 'center', ha='center',fontweight='light',
                       color='#4a4a4a')

    for s in ['top', 'left', 'right']:
        ax.spines[s].set_visible(False)

    fig.text(0.1, 0.95, title, fontproperties=fontprop)    
    ax.grid(axis='y', linestyle='-', alpha=0.4)    
    plt.show()


In [ ]:
df_year_19yrs = df_year[:20].sort_index(ascending=True)

draw_bar_chart(df_year_19yrs, '연도별 출시 타이틀 수')

영화와 TV시리즈의 편수를 보면

In [ ]:
df_type = df_all['type'].value_counts()

for index in df_type.index:
    print(f'{index}\t: {df_type[index]}편')
    
plt.pie(df_type.values, labels=df_type.index, autopct='%.1f%%')
plt.show()
    

이번에는 감독에 대해서 알아보면, 감독 데이터를 추출하면 아래와 같은데

감독 정보가 없는 경우도 있어서 타이틀 수와 감독 수가 일치하지 않음, 물론 같은 감독이 여러 작품을 만드는 경우도 있고

In [ ]:
df_director = df_all['director'].dropna().value_counts()

df_director

다작 감독 10위까지만 뽑아 보면

In [ ]:
df_director_ten = df_director[:10]


for index in df_director_ten.index:
    print(f'{index} : {df_director_ten[index]}편')

스콜세이지, 스필버그 감독이야 유명하니까 알겠지만

1등을 한 Raúl Campos, Jan Suter는 처음 보는 감독들이어서, drill down해 보면(공통 연출은 감독 한 명으로 간주함)

In [ ]:
df_raul_list = df_all[df_all['director'] == 'Raúl Campos, Jan Suter']

df_raul_list.head()

... 그래도 모르겠음. 대충 멕시코에서 스탠드업 코메디를 만들었다는 정도.

감독은 이제 봤으니까, 만들어진 국가를 알아 보면

In [ ]:
def split_data(data):
    buff = {}
    
    for i in range(len(data)):
        if type(data[i]) == str:
            for j in data[i].split(','):
                item = j.strip()

                if item in buff:
                    buff[item] +=1
                else:
                    buff[item] = 1

    index = np.array([])
    values = np.array([])
    
    for item in sorted(buff.items(), reverse=True, key=lambda item: item[1]):
        index = np.append(index, item[0])
        values = np.append(values, item[1])
    
    return pd.Series(data=values, index=index, dtype=np.int)

In [ ]:
data = split_data(df_all['country'])

data

상위 10개국만 추려서 바차트로 그려 보면

당연히 미국이 1위고,발리우드 2위고 우리나라도 8위로 순위권임

In [ ]:
draw_bar_chart(data[:10], '만들어진 나라 상위 10')

출연한 배우들을 살펴 보면

In [ ]:
data = split_data(df_all['cast'])

data

1위는 아누팜 케르라고 함. 잘 모르겠음. 아래와 같은 분이라고 함

![](https://lh3.googleusercontent.com/proxy/vlrk6KjNc1gVI1-xUDKmOqK0sSe6mBGEze5JYwOtBHD8ymjY511rMDIrvFfcm469052gYMBSVkjGiI1RgFNYgMfo_zdK4f1rV8R0MH0mUsGjuVZWAcRj5Z_H5lDhb2PkxHpUsVGUqLbxHW7ZiVEE)

역시 10위까지 차트를 그려보면

In [ ]:
draw_bar_chart(data[:10], '최다 출연 배우 10위')

장르, 러닝 타임, 시청제한 연령 대도 더 남았지만, 너무 많이 알아봐서 이제 그만 알아보기로 함.

끝~